# Introduction to Deep Learning: Final Project
## Introduction
During this class, we touched upon many topics in the field of Deep Learning, and learned about neural networks, stochastic gradient descent, and other techniques and approaches to applying Deep Learning to real world problems.

The subject that I was - and still am - most excited by was last week's module on Generative Adversarial Networks (GANs). In particular, the ability for a generator and discriminator to work together to "learn" the style of source material. For the Final Project, I wanted to apply that knowledge gained more in-depth to another area of art. Both as a means to better understand how to develop GANs, as well as to explore the fun - and often surprising - output of these systems.

My Final Project will be to train a GAN on the data of an artist, in this case [Pablo Picasso](https://en.wikipedia.org/wiki/Pablo_Picasso), a painter whose [cubist](https://en.wikipedia.org/wiki/Cubism) style popularized an entirely new approach and way of thinking about art.

Specifically, I have always been fond of Picasso's attempts at portraits, which is where I think cubism really shines. Therefore, **the goal of my Final Project will be to train a GAN on a collection of Picasso's art, in order to produce a model which can transform photos of people's faces into a cubist style.**

The code for this project can be found at: https://github.com/buffs28349/IntroDeepLearningFinalProject

In order to improve training time, and get more experience using GPUs, I would suggest running this notebook on [Google Colab](https://colab.research.google.com/), a platform for running Jupyter Notebooks in your browser, targeted towards scientific computing and collaboration. You can find a link to this notebook on Colab here: https://colab.research.google.com/drive/1WsVF43K9L31k3GEiRUgYJXGRQLUXGdti?usp=sharing

A zip file of the collection of Picasso generated GAN portraits can be found at this Google Drive link: https://drive.google.com/file/d/1B-i7GaSIUSmosu11pRRvvD8WBPkdPkqh/view?usp=sharing There are a half-dozen samples contained in the GitHub repository

First, let's begin by importing the necessary libaries:

In [ ]:
# In addition to the usual libraries like Tensorflow/Keras
# make sure that these are installed as well
!pip install tensorflow_addons
!pip install pillow

In [ ]:
# General (EDA, Plotting, etc)
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
import PIL
import shutil

# Tensorflow/Keras for GAN work
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

# Define some useful constants
RAND_SEED = 123
EPOCHS = 65 # Will take a while with a CPU, try using a GPU on Google Colab
IMAGE_SIZE = [256, 256]
OUTPUT_CHANNELS = 3

Lastly, since I am using Google Colab for its GPUs, but you may be running this notebook locally, we need to check the environment we are in, in order to be able to locate the data in the next section.

In [ ]:
try:
  from google.colab import drive
  drive.mount("/content/drive")
  IN_COLAB = True
except:
  IN_COLAB = False

DATA_PATH = "/content/drive/My Drive/picasso_gan_final_data" if IN_COLAB else "./data"

### Acquiring the Data
After digging through Kaggle's open datasets, I found a gem. [This dataset](https://www.kaggle.com/datasets/techiewaynezheng/picassowikiart256x256) contains 256x256 images, just like last week's GAN data, that was helpfully uploaded by a Kaggle user from [WikiArt](https://www.wikiart.org/). Although it is not necessarily best practice, I have added these images into the GitHub repository for my Final Project, so that my peers may more easily follow along; they are located in the `data/` directory.

Additionally, I am using a subset of the [CelebA dataset](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset) as the portraits to be used to generate Picasso-style cubist images from. The dataset is huge - we are only using a small fraction of it.

In [ ]:
PICASSO_FILES = glob.glob(f"{DATA_PATH}/picasso/*.jpg")
PORTRAIT_FILES = glob.glob(f"{DATA_PATH}/portrait/*.jpg")

def decode_image(image_filename):
  image_string = tf.io.read_file(image_filename)
  image_decoded = tf.image.decode_jpeg(image_string, channels = 3)
  image = (tf.cast(image_decoded, tf.float32) / 127.5) - 1
  return image

def load_data(filenames):
    ds = tf.data.Dataset.from_tensor_slices(filenames)
    ds = ds.map(decode_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
    return ds

picasso_data = load_data(PICASSO_FILES).batch(1)
portrait_data = load_data(PORTRAIT_FILES).batch(1)

## Exploratory Data Analysis (EDA)
We are developing a GAN to generate images from a training set (Picasso's paintings). In the previous **Introduction** section we loaded our dataset into `picasso_data` and `portrait_data`. We can imagine what this data might be, but it is still important as part of exploratory data analysis to be certain and take a look. First, `picasso_data`:

In [ ]:
picasso_sample_image = next(iter(picasso_data))
plt.subplot(111)
plt.title('Picasso Sample Image')
plt.imshow(picasso_sample_image[0] * 0.5 + 0.5)

Picasso was truly ahead of his time! This is a great example of what we expect: *a 256x256 pixel Picasso cubist painting.* Now one of the portraits that we will use:

In [ ]:
portrait_sample_image = next(iter(portrait_data))
plt.subplot(111)
plt.title('Portrait Sample Image')
plt.imshow(portrait_sample_image[0] * 0.5 + 0.5)

A portrait photo of a person - just what we expected! If it isn't already obvious, we will be training on the images in `picasso_data` and utilizing the GAN to generate Picasso-style cubist images from the source photos in `portrait_data`. The portraits, too, are 256x256 pixels.

To get a feel for the amount of training data we have, and the number source images we must generate, let's take a look at the size of both `picasso_data` and `portrait_data`, each of which were loaded from raw images into a Tensorflow [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
num_picasso = len(PICASSO_FILES)
num_portrait = len(PORTRAIT_FILES)

print(f"There are {num_picasso} Picasso training images.")
print(f"There are {num_portrait} portrait target images.")

plt.subplot(111)
plt.bar(["Picasso", "Portrait"], [num_picasso, num_portrait], color = 'green', width = 0.5)
plt.xlabel("Dataset")
plt.ylabel("Number of Images")
plt.title("Total Count of Images in Picasso and Portrait Datasets")
plt.show()

I made sure to collect a strong quantity of both Picasso images - thanks to WikiArt - as well as a large number of potential portraits (albeit not as many as the entire original dataset contained).

This allows us to both have a sufficient quantity of training data (1169 Picasso paintings) as well as a sufficient quantity of portraits to generate from (also 1169).

Now on to model building and training!

## Analysis (Model Building + Training)
We will be utilizing a [Deep Convolutionl Generative Adversarial Network](https://www.tensorflow.org/tutorials/generative/dcgan) (DCGAN) for our model. We will be using this because Tensorflow has strong support for this type of GAN, and I thoroughly enjoed learning about and developing with it during Week 5.

I was inspired by the code [from here](https://www.kaggle.com/code/amyjang/monet-cyclegan-tutorial/notebook). However, I improved on the process myself for a more performant and effective training procedure specific to the requirements for our mini-project this week. ***The work is entirely my own.***

### Generator
The first part of a GAN is of course the generator. We start by creating helper functions, `downsample` and `upsample` to be used when creating the generator.

Our generator consists of first downsampling our Picasso painting images, before upscaling them back to the required resolution (256x256). This CNN (the "DC" in DCGAN) process allows us to more thoroughly iterate on the learning procedure, providing many benefits - most notably, a better loss value and therefore enhanced output quality.

In [ ]:
def downsample(filters, size, apply_instancenorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
        kernel_initializer=initializer, use_bias=False))

    if apply_instancenorm:
        result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    result.add(layers.LeakyReLU())

    return result


def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2,
        padding='same',
        kernel_initializer=initializer,
        use_bias=False))

    result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    if apply_dropout:
        result.add(layers.Dropout(0.5))

    result.add(layers.ReLU())

    return result

After defining our `downsample` and `upsample` functions we can use them to assemble the layers of our generator:

In [ ]:
def make_generator():
    inputs = layers.Input(shape=[256,256,3])

    down_stack = [
        downsample(64, 4, apply_instancenorm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
    ]

    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
        strides=2,
        padding='same',
        kernel_initializer=initializer,
        activation='tanh')

    x = inputs

    # Perform downsampling
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    # Perform upsampling
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    x = last(x)

    return keras.Model(inputs=inputs, outputs=x)
    
generator = make_generator()

Let's view the generator's summary to confirm it produced the layers that we expect:

In [ ]:
generator.summary()

### Discriminator
Next is the discriminator. It is similar to the generator but without upsampling. Essentially our layers will continuously refine its understanding of the image before being used to make a decision as to whether the image is a real Picasso or a fake: binary classification.

In [ ]:
def make_discriminator():
    initializer = tf.random_normal_initializer(0.0, 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    inp = layers.Input(shape=[256, 256, 3], name='input_image')

    x = inp

    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)

    zero_pad1 = layers.ZeroPadding2D()(down3)
    conv = layers.Conv2D(512, 4, strides=1,
        kernel_initializer=initializer,
        use_bias=False)(zero_pad1)

    norm1 = tfa.layers.InstanceNormalization(gamma_initializer=gamma_init)(conv)

    leaky_relu = layers.LeakyReLU()(norm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)

    last = layers.Conv2D(1, 4, strides=1,
        kernel_initializer=initializer)(zero_pad2)

    return tf.keras.Model(inputs=inp, outputs=last)

discriminator = make_discriminator()

And the discriminator's summary:

In [ ]:
discriminator.summary()

**Important Note:** Since we have not yet trained the generator, its output will be nothing of value to us. This is demonstrated below:

In [ ]:
to_picasso = generator(portrait_sample_image)

plt.subplot(121)
plt.title("Original")
plt.imshow(portrait_sample_image[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title("Picasso Cubist")
plt.imshow(to_picasso[0] * 0.5 + 0.5)
plt.show()

### Training
Now this is where the magic happens. We subclass `keras.Model` and create our own `PicassoGAN` from it. Specifically, we will override the `train_step` to better track the quality of our data during fitting, as well as customize the generator/discriminator relationship of the GAN itself.

In [ ]:
class PicassoGAN(keras.Model):
  def __init__(self, generator, discriminator,):
    super(PicassoGAN, self).__init__()
    self.generator = generator
    self.discriminator = discriminator
        
  def compile(self, generator_optimizer, discriminator_optimizer, gen_loss_fn, disc_loss_fn):
    super(PicassoGAN, self).compile()
    self.generator_optimizer = generator_optimizer
    self.discriminator_optimizer = discriminator_optimizer
    self.gen_loss_fn = gen_loss_fn
    self.disc_loss_fn = disc_loss_fn

  # Necessary for saving model
  # Source: https://stackoverflow.com/a/69460770
  def call(self, inputs, training=False):
    self.weight_dict['backbone'].trainable = False
    x = self.weight_dict['backbone'](inputs)
    x = self.weight_dict['outputs'](x)
    return x
        
  def train_step(self, batch_data):
    real_picasso, real_portrait = batch_data
        
    with tf.GradientTape(persistent=True) as tape:
      # Generate fake Picassos from input portraits
      fake_picasso = self.generator(real_portrait, training=True)
            
      # Discriminator
      disc_real_picasso = self.discriminator(real_picasso, training=True)
      disc_fake_picasso = self.discriminator(fake_picasso, training=True)

      # Generator loss
      gen_loss = self.gen_loss_fn(disc_fake_picasso)
            
      # Discriminator loss
      disc_loss = self.disc_loss_fn(disc_real_picasso, disc_fake_picasso)

    # Calculate gradients
    generator_gradients = tape.gradient(gen_loss, self.generator.trainable_variables)
    discriminator_gradients = tape.gradient(disc_loss, self.discriminator.trainable_variables)
        
    # Apply gradients to optimizer
    self.generator_optimizer.apply_gradients(zip(generator_gradients, self.generator.trainable_variables))
    self.discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.discriminator.trainable_variables))
        
    return {
      "gen_loss": gen_loss,
      "disc_loss": disc_loss,
    }

Lastly, we must define our loss functions. These compare the real images and fake images. A "perfect" discriminator will produce all correct, or true values (that is `1`s) and a poor generator all incorrect values (`0`s).

It has already been said, but is worth repeating: the incredible power of a GAN is in combining these together so that the discriminator improves the generator by identifying real vs fakes, thus requiring the generator to improve itself to "trick" the discriminator.

In [ ]:
def discriminator_loss(real, generated):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(real), real)
    generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss * 0.5
    
def generator_loss(generated):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(generated), generated)

Now we may perform the actual training:

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)

picasso_model = PicassoGAN(generator, discriminator)

picasso_model.compile(generator_optimizer, discriminator_optimizer, generator_loss, discriminator_loss)

picasso_model.fit(tf.data.Dataset.zip((picasso_data, portrait_data)), epochs=EPOCHS)

We will save our model so that we don't have to retrain it in the future!

In [ ]:
picasso_model.compute_output_shape(input_shape=(None, 256, 256, 3))
picasso_model.save(f"{DATA_PATH}/picasso.model")

## Results
We have trained our model! Let's see what it can do:

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(12, 12))
for i, img in enumerate(portrait_data.take(3)):
    prediction = generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Real Portrait Input")
    ax[i, 1].set_title("Fake Picasso Output")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")
plt.show()

### Save Output to Share
We will create a dedicated directory, `cubist_portrait_output` to save each portrait to. Plus, we will archive them into a zip file for easily distribution and sharing.

In [ ]:
if not os.path.exists(f"{DATA_PATH}/cubist_portrait_output"):
  os.mkdir(f"{DATA_PATH}/cubist_portrait_output")

i = 1
for img in portrait_data:
    prediction = generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    im = PIL.Image.fromarray(prediction)
    im.save(f"{DATA_PATH}/cubist_portrait_output/{str(i)}.jpg")
    i += 1
    
shutil.make_archive(f"{DATA_PATH}/cubist_portraits", 'zip', f"{DATA_PATH}/cubist_portrait_output")

## Discussion/Conclusion
Despite the generated images from the input portraits not quite looking like Picasso's cubist style, I still consider this Final Project a success. We are able to improve on our understanding of GANs from Week 5 by applying the same EDA and model building methodology to a novel problem (generation of Picasso cubist images). Our loss was acceptable (`< 0.9`), albeit the result did not resemble Picasso.

This knowledge is still useful, because it teaches us that refinement of the layers of our generator and discriminator is warranted. Additionally, tweaking hyperparameters is in order, both to better improve model training efficiency as well as improve model output.

I learned quite a lot in this class, and am especially fond of my learning of GANs. I hope to further expand my Data Science skillset as applied to Machine Learning and Artificial Intelligence.